In [2]:
import pandas as pd
import re
from tqdm import tqdm

In [108]:
cue_list = ['linguistic', 'contextual', 'emotional']
task_name = 'iacv2'

code_pattern  = re.compile(r'^(?: {2,}|\t?|)(?:def|import|from)\b.*$', re.MULTILINE)
code_line_pattern = re.compile(r'\s*###?code:|###?Code:|###?Python:|Python:|code:|python:|###Input:|Input:', re.MULTILINE | re.IGNORECASE)

for dataset_type in ['train','test']:
    df = pd.read_csv(f'llama_toc_datasets_new/{dataset_type}_{task_name}_with_toc_cues.csv')
    for i, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing {dataset_type} {task_name} dataset"):
        for cue_type in cue_list:
            cue = row[cue_type + '_cue']

            if isinstance(cue, str):
                cue = re.sub(r'^\n+', '', cue)
                split_cue = re.split(r'\n\n(?!\n)', cue)
            else: 
                cue_new = cue 
                df.at[i, cue_type + '_cue_processed'] = cue_new  
                continue

            if len(split_cue) > 1:  
                cue_new = split_cue[0] + '\n\n' + split_cue[1]
                cue_new = code_pattern.split(cue_new)[0].strip()
                cue_new = code_line_pattern.split(cue_new)[0].strip()
            else:
                cue_new = cue  
            df.at[i, cue_type + '_cue_processed'] = cue_new  
    
    df.to_csv(f'llama_toc_datasets_new_processed/{dataset_type}_{task_name}_with_toc_cues_processed.csv', index=False)


Processing test iacv2 dataset: 100%|██████████| 1042/1042 [00:00<00:00, 1210.31it/s]
